# Get observations from program ID
This notebook uses the [MAST API](https://mast.stsci.edu/api/v0/) to query specific data products of a specific program. We are looking for NIRISS imaging and WFSS files of the [NGDEEP program](https://www.stsci.edu/jwst/phase2-public/2079.pdf) (ID 2079). The observations are in three NIRISS filters: F115W, F150W, and F200W.

**Use case**: use MAST to download data products.<br>
**Data**: JWST/NIRISS images and spectra from program 2079.<br>
**Tools**: astropy, astroquery, numpy<br>
**Cross-instrument**: all<br>

**Content**
- [Imports](#imports)
- [Set up the query and the batches](#setup)
- [Filter and download products](#filter)


**Author**: Camilla Pacifici (cpacifici@stsci.edu)
**Last modified**: July 2023

This notebook was inspired by the [JWebbinar session about MAST](https://github.com/spacetelescope/jwebbinar_prep/blob/main/mast_session/Crowded_Field/Crowded_Field.ipynb).

<a id='imports'></a>
## Imports

In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astroquery.mast import Observations
import numpy as np

<a id='setup'></a>
## Set up the query and the batches

Select the proposal ID, instrument, and some useful keywords (filters in this case).

In [ ]:
obs_table = Observations.query_criteria(obs_collection=["JWST"], 
                                        instrument_name=["NIRISS/IMAGE", "NIRISS/WFSS"],
                                        provenance_name=["CALJWST"], # Executed observations
                                        filters=['F115W','F150W','F200W'],
                                        proposal_id=[2079],
                                       )
obs_table

I could list all the products, but if there are too many it could time out. Let us devide the observations in baches and then filter and download one batch at a time.

In [ ]:
batch_size = 5
# Let's split up ``obs_table`` into batches according to our batch size.
obs_batches = [obs_table[i:i+batch_size] for i in range(0, len(obs_table), batch_size)]
single_group = obs_batches[0] # Useful to inspect the files obtained from one group
print("How many batches?", len(obs_batches))

<a id='filter'></a>
## Filter and download products

Select the type of products needed. I do one at a time. I need:
- rate images
    - productType=["SCIENCE"]
    - productSubGroupDescription=['RATE']
    - calib_level=[2]
- level 2 associations for both spectroscopy and imaging
    - productType=["INFO"]
    - productSubGroupDescription=['ASN']
    - calib_level=[2]
- level 3 associations for imaging
    - productType=["INFO"]
    - productSubGroupDescription=['ASN']
    - dataproduct_type=["image"]
    - calib_level=[3]

In [ ]:
# Now let's get the products for each batch of observations, and filter down to only the products of interest.
for index, batch in enumerate(obs_batches):
#for index, batch in enumerate(single_group): # Useful to inspect the files obtained from one batch
    
    # Progress indicator...
    print('\n'+f'Batch #{index+1}')
    
    # Make a list of the `obsid` identifiers from our Astropy table of observations to get products for.
    obsids = batch['obsid']
    print('Working with the following ``obsid``s:')
    print(list(obsids))
    
    # Get list of products 
    products = Observations.get_product_list(obsids)
    
    # Filter the products to only get science products of calibration level 3 
    filtered_products = Observations.filter_products(products, 
                                                     productType=["INFO"],
                                                     productSubGroupDescription=['ASN'],
                                                     dataproduct_type=["image"],
                                                     calib_level=[3]
                                                    )
    # Download products for these records.
    print('Products downloaded:')
    print(filtered_products['productFilename'])
    manifest = Observations.download_products(filtered_products)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>